# Introduction to CNN and TensorFlow

This tutorial gives a very brief introduction of Convolutional Neural Network (CNN) and TensorFlow. Comparing with existing tutorials on TensorFlow and CNN, which are enormous, this tutorial focuses more on the mechanism behind TensorFlow. We elaborate based on the concept of computation graph, which TensorFlow uses to define, execute, and restore computation, not only for CNN, but also for general tasks.

This tutorial is organised as follows:

1. [Basic TensorFlow usage](#Basic-TensorFlow-Usage)
2. Computaion Graph in TensorFlow
3. CNN as computation graph

Our tutorial is targeting at the **graph execution** mode of TensorFlow, in contrast to **eager execution**, which is introduced by recent TensorFlow releases to perform computation on-the-fly, without building the computation graph at first.

Only the **inference** task is covered in this tutorial.

In the following tutorials, we will feed the TensorFlow models we build and store here for further processing.

## Basic TensorFlow Usage

TensorFlow is, on the one hand, a famous machine learning framework on which developers can easily build their models, while on the other hand, a **Domain-Specific Language** (DSL) embedded in Python. As a DSL, TensorFlow provides *primitives*, or APIs, to construct computation graphs for machine learning models. Therefore, in this tutorial, we introduce TensorFlow as a programming language: we first present elements of a typical programming language from TensorFlow, such as syntax, variables, data types, literals, and operators; next, show how a program constructed by TensorFlow can be executed; in the end, there will be a short list of tips and tricks about using TensorFlow.

We will look into the mechanism of computation graph in TensorFlow in the following sections.

### Basic Syntax

Assuming you have already installed TensorFlow (if not, please follow this [tutorial](https://www.tensorflow.org/install/)), the last step before using TensorFlow is importing it in Python.

In [1]:
import tensorflow as tf

/home/rz3515/projects/corerain/papaa18-tutorial/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Computation Graph in TensorFlow

## CNN as Computation Graph